# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [19]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [21]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [23]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## explanation: 
* what this piece of code does: links = [link.get('href') for link in soup.find_all('a')]
* This piece of code is used to extract all the links from HTML content parsed by a library like BeautifulSoup in Python. Here's a breakdown of what it does:

1. **`soup.find_all('a')`**: This part of the code uses BeautifulSoup to find all HTML elements with the tag `'a'`, which typically represent hyperlinks.

2. **`link.get('href')`**: For each `'a'` tag found, this extracts the value of the `'href'` attribute, which contains the actual URL of the link.

3. **`[link.get('href') for link in soup.find_all('a')]`**: This is a list comprehension that iterates over all the `'a'` tags found and creates a list of their `'href'` values.

So, the variable `links` will end up being a list of URLs found in the HTML content.

Here's a more detailed explanation with an example:

### Example

Suppose you have the following HTML content:

```html
Example
Google
```

If you parse this content with BeautifulSoup and run the given code, `links` would be:

```python
links = [
    'https://example.com',
    'https://google.com'
]
```

### Full Example Code

Here's how you might use this in a full script:

```python
from bs4 import BeautifulSoup
import requests

# Fetch HTML content
url = "https://example.com"
response = requests.get(url)

# Parse HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract links
links = [link.get('href') for link in soup.find_all('a')]

# Print extracted links
for link in links:
    print(link)
```

This code fetches HTML from a URL, parses it, extracts all links, and then prints them.

In [29]:
ed = Website("https://www.figure.ai/")
ed.links

['/',
 '/about-us',
 '/ai',
 '/master-plan',
 '/culture',
 '/news',
 '/careers',
 'https://www.youtube.com/watch?v=Z3yQHYNXPws',
 'https://twitter.com/figure_robot',
 'https://www.instagram.com/figure_robot/',
 'https://www.youtube.com/@figureai',
 'https://www.linkedin.com/company/figure-ai/',
 '/master-plan',
 '/about-us',
 '/ai',
 '/careers',
 'https://twitter.com/figure_robot',
 'https://www.instagram.com/figure_robot/',
 'https://www.linkedin.com/company/figure-ai/',
 'https://www.youtube.com/@figureai',
 '/about-us',
 '/ai',
 '/master-plan',
 '/culture',
 '/news',
 '/careers',
 '/terms-and-conditions',
 '/privacy-policy',
 '/accessibility-statement',
 'https://policies.google.com/privacy',
 'https://policies.google.com/terms']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [23]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [15]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [19]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [33]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/claude-3-7-sonnet',
 '/claude/sonnet',
 '/news/visible-extended-thinking',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/solutions/coding',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosu

In [21]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/coding'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [23]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [31]:
Website("https://www.figure.ai/").get_contents()

'Webpage Title:\nFigure\nWebpage Contents:\nAbout Us\nAI\nMaster Plan\nCulture\nNews\nCareers\nFigure is the first-of-its-kind AI robotics company bringing a general purpose humanoid to life.\nWatch the full video\nFIGURE IS\nFINALLY\nBRINGING ROBOTS\nHOME.\nFIGURE IS\nPUTTING\nLEARNING ROBOTS\nINTO THE GLOBAL WORKFORCE.\nFIGURE IS\nTAKING AN\nAI-FIRST APPROACH\nTO MARKET.\nFIGURE IN THE WORKFORCE\nFIGURE AT\nHOME\nWe’re engineering the first humanoid that can truly think  for itself.\nFigure is poised to solve labor shortages. Up and running for our first commercial customers, it’s learning and performing end-to-end operations safely. Now, in the biggest innovation since the dishwasher, Figure is finally coming home. Expect Figure to do dishes and laundry, unpack groceries, and perform other household tasks 100% autonomously.\nSee our Master Plan\nWE’RE GIVING AI A BODY.\nDesigned for our world\nWhy a human form factor?\nWe’ve designed our world for the human form. Hands allow us to o

In [39]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Announcement
Claude 3.7 Sonnet and Claude Code
Introducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.
Read 

In [25]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for children. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. \
Make the brochure look humorous and attractive for children who we want to encourage to join in part time jobs. \
include a lot of emojis"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20000] # Truncate if more than 20,000 characters
    return user_prompt

In [29]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nAPI\nResearch\nCommitments\nLearn\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.7 Sonnet\nClaude 3.7 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nClaude 3.7 Sonnet and Claude Code\nIntroducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.\nRead the announcement\nModel details\nClaude 3.7 Sonnet\nResearch insights\nClaude\'s extended thinking\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [37]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}]}


```markdown
# 🎉 Welcome to Anthropic: The AI Playground! 🌟

Hey there, young explorers! 🌈 Are you ready to dive into the magical world of AI? At Anthropic, we're shaping the future with safety, fun, and a sprinkle of magic! ✨ 

---

## 🌟 Who Are We? 🤔

We're a quirky team of researchers, engineers, and creative thinkers based in sunny San Francisco! ☀️ Our mission? To build friendly AI systems that make the world a better place! 🌏💖 We love to learn, and share what we discover about how to keep AI safe for everyone. 🚀

---

## 💼 What's Our Company Culture Like? 🎈

Here at Anthropic, we believe in teamwork and friendship! 🤝 Here’s how we roll:

- **Safety First! 🚦**: We treat AI with the utmost care, like a kitten on a balance beam! 
- **Good Vibes Only! ✌️**: We gather every kind of talent (from rocket scientists to pizza lovers 🎉), because everyone can contribute with their unique skills!
- **Learning is Fun! 📚✨**: Just like exploring the world of video games, we believe in curiosity and growing together! 

---

## 🎈 Our Customers - Who Do We Help? 🦸‍♂️🦸‍♀️

From businesses, nonprofits, to everyday people like you (yes, YOU! 👀), we create cool tools like **Claude** to make sure AI is beneficial and safe! 

---

## 🎤 Join Our Team! 🚀

Thinking about a part-time job? Anthropic is the place for you! Here’s why you should hop on board:

- **Flexible Fun**: We offer flexible hours; you can work on your own schedule! 🕒💼
- **Awesome Benefits**: Snacks, games, and a health pack that would make superheroes jealous! 🍕🍪
- **Real Impact**: Help us make the world a safer place with AI! 🌍

---

## 🔍 What Do We Look For? 💎

When you join us, bring your enthusiasm and uniqueness! Experience isn’t everything; just like in school, it’s about learning and growing together! 🌱

---

## 🦄 Why Work at Anthropic? 🎊
Working with us means you'll:
- Be a superhero in the world of AI!
- Make decisions that really matter! 
- Meet a bunch of awesome friends who share your dreams! 

So what are you waiting for? Your adventure begins here at Anthropic! 🚀💖 

**🌈 Join Us and Let’s Create Magic Together! 🌟**

📩 Want to learn more? Ask your parents to check out our website or contact us at support@anthropic.com! 

---
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [33]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}



# 🐜 Welcome to the Wacky World of Anthropic! 🎉

Hey there, future geniuses! 🤓 Are you ready to dive into the awe-inspiring world of AI and help build a better future? At **Anthropic**, we're on a mission to make AI safe, fun, and reliable! 🎈💡

---

## 🐜 Who Are We? 🤔

We are a team of super-smart researchers, engineers, and policy wizards based in sunny **San Francisco**! 🌞 Our superhero tech, **Claude**, is like the friendly genius of AI. He's here to help everyone from businesses to your mom’s favorite charity! 🏢💖 

---

## 🏆 Our Mission! 🎯

At Anthropic, we believe AI can change the world for the better! 🌍✨ We're committed to creating systems that are safe and helpful. Think of us as the safety helmets for AI 🚧 — making sure it doesn't do anything wacky (like trying to take over the world 😜).

---

## 🌈 What’s It Like to Work Here? 🤗

Our team is more like a big family that loves to help and support each other! 🙌 We have fun, grow together, and even talk about plants (yes, plants)! 🌱 

### 🥳 Benefits & Perks:
- **Health & Wellness:** We take care of our superheroes with awesome health insurance! 🏥
- **Flexible Time Off:** Need some fun time? Go for it! 🎉
- **Snacks Galore:** We have daily meals and snacks in the office! 🍕🍪
- **Learning & Growth:** We love it when our team members learn new things! 📚

---

## 🕵️‍♀️ Who Do We Help? 🤝

From big companies to everyday people 🤵👩, we’re here for ALL our users! Whether you're an adventurous scientist or an eager coder, you can count on **Claude** to help you along the way! 💪💻

---

## 📢 Join the Team! 🚀

Are you excited to make the world a better place with AI? 🌟 We have job openings for smart cookies and eager learners just like you! 

### Open Positions:
- **Junior AI Helpers** – Learn while you earn! 💰
- **Super Coders** – Help us build amazing stuff! 💻
- **Creative Thinkers** – Share your ideas and be part of the excitement! 🎨

Just like Marvel superheroes, we support all types of backgrounds! 🦸 

---

## 🤖 Ready to Join Us? 🎉

Don’t forget: If you have cool ideas or want to help out, **Anthropic** is for YOU! We love superheroes from all walks of life! So, grab your cape and let’s create some amazing AI together! 💥🤩

---

## Contact Us:
Questions? Just give a shout! 📧✉️ [Contact Us](mailto:support.anthropic.com)

---

**Let's be safe, silly, and super together!** 🐜💖🎈


In [59]:
# Try changing the system prompt to the humorous version that encourage children to join the work force at Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🤗 Welcome to Hugging Face! 🤗

Hello, budding geniuses! Are you ready to dive into the **exciting world of Artificial Intelligence (AI)**? At Hugging Face, we’re busy building the future and we want YOU to join our adventure! 🚀✨

---

## 🌟 **Who Are We?** 

We are a super-duper community of tech-savvy pals who create, discover, and collaborate! 🤝 We make cool AI models, datasets, and applications that help people make amazing things happen. It’s like being a superhero, but with technology! 🦸‍♂️ 

---

## 🧑‍🤝‍🧑 **Our Customers!** 

Guess what? Over **50,000 organizations** like Amazon, Microsoft, Instagram, and Google are using our tools! 📈 That means lots of people are having fun with AI. They come here to explore special features and benefit from our friendly community. 🌍❤️

---

## 👩‍💻 **Your Future Superpowers!** 

Ever thought about what it’s like to work in tech? Well, at Hugging Face, we have part-time jobs where you can learn, grow, and have a blast! 🥳 Whether you're into rolling up your sleeves on coding, writing, or creating AI magic, we want you! 

Here’s what you can do:
1. **Create cool models** – The sky is the limit! 
2. **Collaborate with experts** – Meet and work with friendly wizards of AI! 🧙‍♂️
3. **Showcase your skills** – Build your very own ML profile and become an AI star! 🌟

---

## 🎉 **Join the Fun!** 

At Hugging Face, we believe in **teamwork**, **creativity**, and a sprinkle of humor! 😊 Our secret ingredient? A workplace that feels like one big happy family! 💖 

---

## **Why Work With Us?** 🤔

- **Learn Like a Pro!** – Get hands-on experience with real AI projects! 
- **Make New Friends** – Collaborate with super cool people who love AI just like you! 
- **Flexible Hours** – We understand that school is important. Work part-time that fits your schedule! 🌈
- **Epic Rewards** – Not only do you gain skills, but you will also earn a cool paycheck! 💰

---

## 🦄 **Join The AI Party!**

If you’re ready to unleash your creativity and super-charged ideas, head over to our careers page (👩‍💻) and let’s make magic together! 

➡️ **Sign Up Now!** 

---

🎈 So come on, heroes of tomorrow! Let’s build the future with AI together at Hugging Face! ✨ Remember, the world needs more thinkers, creators, and dreamers like YOU! 🌈

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>